In [66]:
import warnings
warnings.filterwarnings("ignore")

In [67]:
import numpy as np
import pandas as pd

In [68]:
train = pd.read_csv("C:/Users/Aravind/PROJECTS/Used-Car-Price-Prediction/Data/train.csv")
test = pd.read_csv("C:/Users/Aravind/PROJECTS/Used-Car-Price-Prediction/Data/test.csv")

In [69]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            54273 non-null  int64 
 1   brand         54273 non-null  object
 2   model         54273 non-null  object
 3   model_year    54273 non-null  int64 
 4   milage        54273 non-null  int64 
 5   fuel_type     54273 non-null  object
 6   engine        54273 non-null  object
 7   transmission  54273 non-null  object
 8   ext_col       54273 non-null  object
 9   int_col       54273 non-null  object
 10  accident      54273 non-null  object
 11  clean_title   54273 non-null  object
 12  price         54273 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 5.4+ MB


In [70]:
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [71]:
train.loc[6]

id                                                          6
brand                                                    Audi
model                                        S5 3.0T Prestige
model_year                                               2014
milage                                                 107380
fuel_type                                            Gasoline
engine          333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel
transmission                                      7-Speed A/T
ext_col                                                  Gray
int_col                                                 Black
accident                                        None reported
clean_title                                               Yes
price                                                   26500
Name: 6, dtype: object

In [72]:
for column in train.select_dtypes(include=['object']).columns:
    print(f"Value counts for column '{column}':")
    print(train[column].value_counts())
    print()

Value counts for column 'brand':
brand
BMW              7369
Ford             6706
Mercedes-Benz    5087
Chevrolet        4424
Audi             2922
Porsche          2627
Toyota           2315
Lexus            2257
Jeep             2247
Land             1995
Cadillac         1565
Nissan           1252
GMC              1076
Dodge            1011
RAM               966
INFINITI          957
Lincoln           767
Mazda             748
Subaru            739
Hyundai           694
Jaguar            661
Volkswagen        628
Honda             624
Acura             580
Kia               526
Volvo             452
MINI              364
Maserati          293
Bentley           284
Chrysler          258
Genesis           249
Buick             228
Mitsubishi        182
Hummer            176
Pontiac           149
Alfa              144
Rolls-Royce       142
Lamborghini       122
Tesla             110
Ferrari            87
Saturn             58
Scion              53
Aston              50
McLaren        

In [73]:
train['int_black'] = train['int_col'].apply(lambda x: 1 if x == 'Black' else 0)
test['int_black'] = test['int_col'].apply(lambda x: 1 if x == 'Black' else 0)

In [74]:
def categorize_color(color):
    color_lower = color.lower()
    if 'black' in color_lower:
        return 'black'
    elif 'white' in color_lower:
        return 'white'
    else:
        return 'other'

train['ext_col_processed'] = train['ext_col'].apply(categorize_color)
test['ext_col_processed'] = test['ext_col'].apply(categorize_color)

In [75]:
# Function to parse the 'engine' column
def parse_engine(engine_str):
    parts = engine_str.split()
    horsepower, displacement, cylinder, engine_type, fuel = (np.nan, np.nan, np.nan, np.nan, np.nan)
    
    try:
        # Parse horsepower
        if 'HP' in parts[0]:
            horsepower = parts[0].replace('HP', '')
            parts.pop(0)
        
        # Parse displacement
        if 'L' in parts[0]:
            displacement = parts[0].replace('L', '')
            parts.pop(0)
        elif 'Liter' in parts[0]:
            displacement = parts[0].replace('Liter', '')
            parts.pop(0)

        # Parse cylinder
        if parts[1] == 'Cylinder':
            cylinder = parts[0]
            parts.pop(0)
            parts.pop(0)
        
        # Parse engine type and fuel
        if len(parts) >= 2:
            engine_type = parts[0]
            fuel = parts[-2] if 'Fuel' in parts[-1] else parts[-1]
    except:
        pass
    
    return pd.Series([horsepower, displacement, cylinder, engine_type, fuel])

# Apply the function to the 'engine' column and create new columns
train[['horsepower', 'displacement', 'cylinder', 'type', 'fuel']] = train['engine'].apply(parse_engine)
test[['horsepower', 'displacement', 'cylinder', 'type', 'fuel']] = test['engine'].apply(parse_engine)

In [76]:
train = train.drop(['id','clean_title','int_col','ext_col','transmission','engine','model','fuel_type'],axis=1)
test = test.drop(['clean_title','int_col','ext_col','transmission','engine','model','fuel_type'],axis=1)

In [77]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   brand              54273 non-null  object
 1   model_year         54273 non-null  int64 
 2   milage             54273 non-null  int64 
 3   accident           54273 non-null  object
 4   price              54273 non-null  int64 
 5   int_black          54273 non-null  int64 
 6   ext_col_processed  54273 non-null  object
 7   horsepower         50216 non-null  object
 8   displacement       53667 non-null  object
 9   cylinder           43555 non-null  object
 10  type               53901 non-null  object
 11  fuel               53901 non-null  object
dtypes: int64(4), object(8)
memory usage: 5.0+ MB


In [78]:
train.head()

,brand,model_year,milage,accident,price,int_black,ext_col_processed,horsepower,displacement,cylinder,type,fuel
0,Ford,2018,74349,None reported,11000,0,other,375.0,3.5,V6,Engine,Gasoline
1,BMW,2007,80000,None reported,8250,1,black,300.0,3.0,NaN,Straight,Gasoline
2,Jaguar,2009,91491,None reported,15000,0,other,300.0,4.2,8,Engine,Gasoline
3,BMW,2022,2437,None reported,63500,0,other,335.0,3.0,NaN,Straight,Hybrid
4,Pontiac,2001,111000,None reported,7850,1,white,200.0,3.8,V6,Engine,Gasoline


In [79]:
# train_cleaned = train.dropna()
# test_cleaned = test.dropna()

In [80]:
# train['model_year'] = train['model_year'].astype('object')
# test['model_year'] = test['model_year'].astype('object')

In [81]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

def preprocess_data(df):
    # Selecting numerical columns
    numerical_columns = df.select_dtypes(include=['number']).columns
    df_numerical = df[numerical_columns]
    
    # Handling missing values for numerical columns using SimpleImputer with mean strategy
    imputer_numerical = SimpleImputer(strategy='mean')
    df_imputed_numerical = pd.DataFrame(imputer_numerical.fit_transform(df_numerical), columns=df_numerical.columns)
    
    # Selecting categorical columns
    categorical_columns = df.select_dtypes(include=['object']).columns
    df_categorical = df[categorical_columns]
    
    # Handling missing values for categorical columns using SimpleImputer with most frequent strategy
    imputer_categorical = SimpleImputer(strategy='most_frequent')
    df_imputed_categorical = pd.DataFrame(imputer_categorical.fit_transform(df_categorical), columns=df_categorical.columns)
    
    # Encoding categorical variables using OrdinalEncoder
    ordinal_encoder = OrdinalEncoder()
    df_encoded = pd.DataFrame(ordinal_encoder.fit_transform(df_imputed_categorical), columns=df_imputed_categorical.columns)
    
    # Merging with non-categorical columns
    df_processed = pd.concat([df_imputed_numerical, df_encoded], axis=1)
    
    return df_processed

# Assuming df is your DataFrame
# Replace 'df' with the name of your DataFrame
train_processed = preprocess_data(train)
test_processed = preprocess_data(test)


In [82]:
train_processed.head()

,model_year,milage,price,int_black,brand,accident,ext_col_processed,horsepower,displacement,cylinder,type,fuel
0,2018.0,74349.0,11000.0,0.0,14.0,1.0,1.0,203.0,21.0,6.0,15.0,9.0
1,2007.0,80000.0,8250.0,1.0,4.0,1.0,0.0,152.0,17.0,5.0,25.0,9.0
2,2009.0,91491.0,15000.0,0.0,21.0,1.0,1.0,152.0,27.0,5.0,15.0,9.0
3,2022.0,2437.0,63500.0,0.0,4.0,1.0,1.0,181.0,17.0,5.0,25.0,10.0
4,2001.0,111000.0,7850.0,1.0,40.0,1.0,2.0,73.0,24.0,6.0,15.0,9.0


In [83]:
# from scipy import stats

# # Calculate the Z-scores of each column
# z_scores = np.abs(stats.zscore(train_processed))

# # Set a threshold for identifying outliers
# threshold = 3

# # Identify the indices of rows containing outliers
# outliers = np.where(z_scores > threshold)
# outlier_indices = set(outliers[0])

# # Drop the rows with outliers
# train_processed = train_processed.drop(outlier_indices).reset_index(drop=True)

In [84]:
train_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   model_year         54273 non-null  float64
 1   milage             54273 non-null  float64
 2   price              54273 non-null  float64
 3   int_black          54273 non-null  float64
 4   brand              54273 non-null  float64
 5   accident           54273 non-null  float64
 6   ext_col_processed  54273 non-null  float64
 7   horsepower         54273 non-null  float64
 8   displacement       54273 non-null  float64
 9   cylinder           54273 non-null  float64
 10  type               54273 non-null  float64
 11  fuel               54273 non-null  float64
dtypes: float64(12)
memory usage: 5.0 MB


In [85]:
train_processed.corr()

,model_year,milage,price,int_black,brand,accident,ext_col_processed,horsepower,displacement,cylinder,type,fuel
model_year,1.000000,-0.625226,0.216150,0.174924,-0.068357,0.216034,0.016653,0.354473,-0.007374,0.027649,0.144609,0.145063
milage,-0.625226,1.000000,-0.248927,-0.164485,0.047288,-0.295166,-0.017469,-0.375689,-0.015869,-0.000990,-0.133896,-0.161784
price,0.216150,-0.248927,1.000000,0.033874,-0.003448,0.114705,-0.004010,0.217910,0.106515,0.026621,0.069234,0.068249
int_black,0.174924,-0.164485,0.033874,1.000000,-0.029687,0.078352,-0.158360,0.122575,-0.002843,-0.019597,-0.028006,0.001279
brand,-0.068357,0.047288,-0.003448,-0.029687,1.000000,-0.012365,-0.000661,-0.137618,-0.069335,-0.012140,-0.173850,0.021246
accident,0.216034,-0.295166,0.114705,0.078352,-0.012365,1.000000,0.001130,0.178829,0.050284,0.034384,0.071904,0.080063
ext_col_processed,0.016653,-0.017469,-0.004010,-0.158360,-0.000661,0.001130,1.000000,-0.027074,-0.011270,-0.030662,-0.037163,-0.045064
horsepower,0.354473,-0.375689,0.217910,0.122575,-0.137618,0.178829,-0.027074,1.000000,0.612704,0.316586,-0.000216,-0.024968
displacement,-0.007374,-0.015869,0.106515,-0.002843,-0.069335,0.050284,-0.011270,0.612704,1.000000,0.299603,-0.062684,-0.157787
cylinder,0.027649,-0.000990,0.026621,-0.019597,-0.012140,0.034384,-0.030662,0.316586,0.299603,1.000000,0.052846,0.012242


In [86]:
correlation_matrix = train_processed.corr()

# Threshold for correlation coefficient
threshold = 0.1

# Get the absolute correlation with the target variable 'price'
target_corr = correlation_matrix['price'].abs()

# Select columns with correlation above the threshold
relevant_columns = target_corr[target_corr > threshold].index

# Assuming 'df' is your original DataFrame
# df = df[relevant_columns]

# Display the relevant columns
print("Relevant columns:", relevant_columns)

Relevant columns: Index(['model_year', 'milage', 'price', 'accident', 'horsepower',
       'displacement'],
      dtype='object')


In [87]:
train_processed = train_processed.drop(['brand','ext_col_processed','cylinder','type','fuel'],axis=1)
test_processed = test_processed.drop(['brand','ext_col_processed','cylinder','type','fuel'],axis=1)

In [88]:
train_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model_year    54273 non-null  float64
 1   milage        54273 non-null  float64
 2   price         54273 non-null  float64
 3   int_black     54273 non-null  float64
 4   accident      54273 non-null  float64
 5   horsepower    54273 non-null  float64
 6   displacement  54273 non-null  float64
dtypes: float64(7)
memory usage: 2.9 MB


In [89]:
train_processed.describe()

,model_year,milage,price,int_black,accident,horsepower,displacement
count,54273.000000,54273.000000,5.427300e+04,54273.000000,54273.000000,54273.000000,54273.000000
mean,2015.091979,72746.175667,3.921844e+04,0.586295,0.735098,165.635546,23.039965
std,5.588909,50469.490448,7.282634e+04,0.492501,0.441285,67.182401,11.577605
min,1974.000000,100.000000,2.000000e+03,0.000000,0.000000,0.000000,0.000000
25%,2012.000000,32268.000000,1.550000e+04,0.000000,0.000000,119.000000,17.000000
50%,2016.000000,66107.000000,2.800000e+04,1.000000,1.000000,154.000000,21.000000
75%,2019.000000,102000.000000,4.500000e+04,1.000000,1.000000,216.000000,31.000000
max,2024.000000,405000.000000,2.954083e+06,1.000000,1.000000,340.000000,59.000000


In [90]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score
import xgboost as xgb

# Split the data into features and target variable
X = train_processed.drop(columns='price')  # Features
y = train_processed['price']  # Target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost model
xgb_reg = xgb.XGBRegressor(random_state=42)

# Train the model
xgb_reg.fit(X_train, y_train)

# Make predictions
y_pred = xgb_reg.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

RMSE: 62077.9607594095


In [91]:
import statsmodels.api as sm

X = train_processed.drop(columns='price') 
y = train_processed['price']

X = sm.add_constant(X)

X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=42)

reg = sm.OLS(y_train , X_train).fit()

reg.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Ordinary least squares
==================================================================================
Model:                   OLS                  Adj. R-squared:         0.082       
Dependent Variable:      price                AIC:                    1025538.5100
Date:                    2024-06-03 17:14     BIC:                    1025598.8085
No. Observations:        40704                Log-Likelihood:         -5.1276e+05 
Df Model:                6                    F-statistic:            608.8       
Df Residuals:            40697                Prob (F-statistic):     0.00        
R-squared:               0.082                Scale:                  5.1229e+09  
----------------------------------------------------------------------------------
                 Coef.       Std.Err.     t     P>|t|      [0.025        0.975]   
----------------------------------------------------------------------------------
const        -2042106.0546 169193.0831 -12.0697 0.0000 -2373728.2667 -1710483.8425
model_year       1027.6655     83.8792  12.2517 0.0000      863.2605     1192.0705
milage             -0.2213      0.0095 -23.3328 0.0000       -0.2399       -0.2027
int_black       -3010.2260    736.1583  -4.0891 0.0000    -4453.1128    -1567.3393
accident         5655.0815    843.2198   6.7065 0.0000     4002.3518     7307.8111
horsepower        117.8334      7.7689  15.1674 0.0000      102.6062      133.0605
displacement      210.6938     41.2334   5.1098 0.0000      129.8754      291.5123
----------------------------------------------------------------------------------
Omnibus:               100984.965         Durbin-Watson:            1.989         
Prob(Omnibus):         0.000              Jarque-Bera (JB):         1654148315.631
Skew:                  26.956             Prob(JB):                 0.000         
Kurtosis:              989.112            Condition No.:            42377943      
==================================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly
specified.
[2] The condition number is large, 4.24e+07. This might indicate
that there are strong multicollinearity or other numerical
problems.
"""

In [92]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb

def evaluate_models(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the models
    models = {
        'Linear Regression': LinearRegression(),
        'Decision Tree': DecisionTreeRegressor(),
        'Random Forest': RandomForestRegressor(),
        'Gradient Boosting': GradientBoostingRegressor(),
        'AdaBoost': AdaBoostRegressor(),
        'K-Nearest Neighbors': KNeighborsRegressor(),
        'Neural Network': MLPRegressor(max_iter=1000),
        'XGBoost': xgb.XGBRegressor(),
        'LightGBM': lgb.LGBMRegressor()
    }

     # Train and evaluate each model
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        r2 = r2_score(y_test, y_pred)
        print(f'{name}: RMSE = {rmse:.4f}, R² = {r2:.4f}')

X = train_processed.drop(columns='price') 
y = train_processed['price']
evaluate_models(X, y)

Linear Regression: RMSE = 49824.8795, R² = 0.1552
Decision Tree: RMSE = 92933.9809, R² = -1.9392
Random Forest: RMSE = 62244.8330, R² = -0.3185
Gradient Boosting: RMSE = 50016.0411, R² = 0.1487
AdaBoost: RMSE = 89832.8429, R² = -1.7463
K-Nearest Neighbors: RMSE = 58215.5185, R² = -0.1533
Neural Network: RMSE = 49154.1622, R² = 0.1778
XGBoost: RMSE = 62077.9608, R² = -0.3114
LightGBM: RMSE = 50480.9763, R² = 0.1328


In [93]:
model = lgb.LGBMRegressor()
X = train_processed.drop(columns='price') 
y = train_processed['price']
model.fit(X, y)

LGBMRegressor()

In [94]:
test_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            36183 non-null  float64
 1   model_year    36183 non-null  float64
 2   milage        36183 non-null  float64
 3   int_black     36183 non-null  float64
 4   accident      36183 non-null  float64
 5   horsepower    36183 non-null  float64
 6   displacement  36183 non-null  float64
dtypes: float64(7)
memory usage: 1.9 MB


In [95]:
test = test_processed.drop(['id'],axis=1)
y_pred = model.predict(test)

In [96]:
y_pred

array([24101.32416689, 17017.78008578, 26188.73972682, ...,
       10535.99396057, 56876.59092326, 11858.24660996])

In [97]:
test_processed['id']

0        54273.0
1        54274.0
2        54275.0
3        54276.0
4        54277.0
          ...   
36178    90451.0
36179    90452.0
36180    90453.0
36181    90454.0
36182    90455.0
Name: id, Length: 36183, dtype: float64

In [98]:
result_df = pd.DataFrame({'id': test_processed['id'], 'price': y_pred})

# Save the result DataFrame to a CSV file
result_df.to_csv('predictions.csv', index=False)

In [99]:
from sklearn.model_selection import GridSearchCV
lgbm = lgb.LGBMRegressor()

# Define the parameter grid
param_grid = {
    'num_leaves': [31, 50, 70],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 5, 7]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

X = train_processed.drop(columns='price') 
y = train_processed['price']

# Fit the model
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Get the best model
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(test)

Best parameters found: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'num_leaves': 31}


In [100]:
result_df = pd.DataFrame({'id': test_processed['id'], 'price': y_pred})

# Save the result DataFrame to a CSV file
result_df.to_csv('better_predictions.csv', index=False)

In [101]:
from sklearn.ensemble import StackingRegressor

stacking_reg = StackingRegressor(
    estimators = [
        ('Extra Trees ',xgb.XGBRegressor()),
        ('Random Forest ',lgb.LGBMRegressor()),
        ('Linear Regression', GradientBoostingRegressor())
    ],
    final_estimator = lgb.LGBMRegressor() , cv = 5
)
stacking_reg.fit(X , y)
y_pred = stacking_reg.predict(test)

# rmse = np.sqrt(mean_squared_error(y_test , y_pred))
# print(f"Root mean squared error of the model : {rmse} ")
# r2 = r2_score(y_test , y_pred)
# print(f"R2 score of the model : {r2} ")

In [102]:
result_df = pd.DataFrame({'id': test_processed['id'], 'price': y_pred})

# Save the result DataFrame to a CSV file
result_df.to_csv('stack_predictions.csv', index=False)